In [135]:
import torch
import torch.nn as nn

class CustomTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers):
        super(CustomTransformer, self).__init__()
        # Define layers for your custom transformer
        self.transformer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = self.fc(x)
        x = self.relu(x)
        return x

# Assuming BERT embeddings size is 768
input_dim = 768
hidden_dim = 100
num_heads = 4
num_layers = 2

# Instantiate your custom transformer model
custom_transformer = CustomTransformer(input_dim, hidden_dim, num_heads, num_layers)






/opt/homebrew/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [137]:
data = list(pd.read_csv("SEC-CompanyTicker.csv",index_col=0).companyName[:200])

bert_embeddings = Embeddings().get_embeddings(data)[1].last_hidden_state[:, 0]
count = 0
# Print the same output multiple times
for _ in range(1000):
    if count % 100 == 0: 
        print(count)
    # Use the transformer to generate output
    output = custom_transformer(bert_embeddings).cpu().detach().numpy()
    count += 1
output

here
0
here
100
here
200
here
300
here
400
here
500
here
600
here
700
here
800
here
900


array([[0.17985766, 0.37838352, 0.39681453, ..., 0.        , 0.        ,
        0.        ],
       [0.03520419, 0.4522131 , 1.0708578 , ..., 0.        , 0.        ,
        0.        ],
       [0.07934731, 0.31097317, 0.6422863 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.30751058, 0.91021216, 0.66185975, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.8194413 , 1.2174973 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.5526661 , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [140]:
data

['Apple Inc.',
 'Microsoft Corp',
 'Alphabet Inc.',
 'Amazon Com Inc',
 'Nvidia Corp',
 'Tesla, Inc.',
 'Berkshire Hathaway Inc',
 'Meta Platforms, Inc.',
 'Eli Lilly & Co',
 'Visa Inc.',
 'Taiwan Semiconductor Manufacturing Co Ltd',
 'Exxon Mobil Corp',
 'Unitedhealth Group Inc',
 'Walmart Inc.',
 'Novo Nordisk A S',
 'Jpmorgan Chase & Co',
 'Spdr S&P 500 Etf Trust',
 'Johnson & Johnson',
 'Mastercard Inc',
 'Lvmh Moet Hennessy Louis Vuitton',
 'Procter & Gamble Co',
 'Broadcom Inc.',
 'Latam Airlines Group S.A.',
 'Home Depot, Inc.',
 'Chevron Corp',
 'Oracle Corp',
 'Merck & Co., Inc.',
 'Abbvie Inc.',
 'Coca Cola Co',
 'Adobe Inc.',
 'Toyota Motor Corp/',
 'Pepsico Inc',
 'Costco Wholesale Corp /New',
 'Asml Holding Nv',
 'Bank Of America Corp /De/',
 'Cisco Systems, Inc.',
 'Alibaba Group Holding Ltd',
 'Salesforce, Inc.',
 'Shell Plc',
 'Astrazeneca Plc',
 'Novartis Ag',
 'Mcdonalds Corp',
 'Accenture Plc',
 'Thermo Fisher Scientific Inc.',
 'Mexican Economic Development Inc',
 '

In [76]:
# Print requires_grad status of parameters
for name, param in custom_transformer.named_parameters():
    print(name, param.requires_grad)


transformer.self_attn.in_proj_weight False
transformer.self_attn.in_proj_bias False
transformer.self_attn.out_proj.weight False
transformer.self_attn.out_proj.bias False
transformer.linear1.weight False
transformer.linear1.bias False
transformer.linear2.weight False
transformer.linear2.bias False
transformer.norm1.weight False
transformer.norm1.bias False
transformer.norm2.weight False
transformer.norm2.bias False
transformer_encoder.layers.0.self_attn.in_proj_weight False
transformer_encoder.layers.0.self_attn.in_proj_bias False
transformer_encoder.layers.0.self_attn.out_proj.weight False
transformer_encoder.layers.0.self_attn.out_proj.bias False
transformer_encoder.layers.0.linear1.weight False
transformer_encoder.layers.0.linear1.bias False
transformer_encoder.layers.0.linear2.weight False
transformer_encoder.layers.0.linear2.bias False
transformer_encoder.layers.0.norm1.weight False
transformer_encoder.layers.0.norm1.bias False
transformer_encoder.layers.0.norm2.weight False
transf

In [ ]:
embeddings

In [73]:
# Use BERT embeddings as input to your transformer
output = custom_transformer(bert_embeddings)

# Your model will now always output the same result, as the parameters are frozen
embeddings_dataset = output.cpu().detach().numpy()
embeddings_dataset

array([[0.32492712, 0.        , 0.        ],
       [0.68284047, 0.        , 0.23811845],
       [0.5894346 , 1.0451578 , 0.        ],
       [0.25341946, 0.        , 1.0140357 ]], dtype=float32)

In [24]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
import faiss

from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class Embeddings:
    #CLS is a special classification token and the last hidden state of BERT Embedding
    def cls_pooling(self, model_output):
        return model_output.last_hidden_state[:, 0]

    #BERT tokenizer of input text
    def get_embeddings(self, text_list):
        encoded_input = tokenizer(
            text_list, padding=True, truncation=True, return_tensors="pt"
        )
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        model_output = model(**encoded_input)
        return self.cls_pooling(model_output).cpu().detach().numpy(), model_output
    
    
    #convert dataset into embeddings dataset to run FAISS
    def makeEmbeddings(self,dataset):
        embeddings = []
        for data in dataset:
            embeddings.append(self.get_embeddings(data)[0])
        return np.array(embeddings)
    
    def getQueryEmbedding(self, query):
        return self.get_embeddings([query])

In [107]:

    
class Faiss:
    def __init__(self):
        pass

    def faiss(self,xb):
        d = xb[0].size
        M = 32
        index = faiss.IndexHNSWFlat(d, M)            
        index.hnsw.efConstruction = 40         # Setting the value for efConstruction.
        index.hnsw.efSearch = 16               # Setting the value for efSearch.
        index.add(xb)
        return index
    
    def query(self,index,xq,k=3):
        D, I = index.search(xq, k)   
        return D, I

In [124]:
embeddings_dataset = output[:len(output)-1]
xq = output[len(output)-1:]

In [173]:
bert_embeddings = Embeddings().get_embeddings(["JP Morgan"])[1].last_hidden_state[:, 0]
xq = custom_transformer(bert_embeddings).cpu().detach().numpy()
xq

array([[0.        , 1.1690228 , 0.38636065, 0.5957296 , 0.        ,
        0.        , 0.06849277, 0.        , 0.        , 0.        ,
        0.7197747 , 0.42237192, 0.        , 0.34231558, 0.05226898,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.6190933 , 0.        , 1.198056  ,
        0.        , 0.        , 0.7734386 , 0.        , 0.        ,
        0.07963869, 0.06494647, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.214389  , 0.        , 0.28094807,
        0.        , 0.        , 0.33917654, 1.3715205 , 0.        ,
        0.        , 0.8025969 , 0.57968336, 0.        , 0.6572527 ,
        0.        , 0.86783373, 0.06633472, 0.76830876, 0.        ,
        0.        , 0.3967817 , 0.15956374, 0.20083565, 0.20332602,
        0.1859    , 0.28096327, 0.4797761 , 0.8370653 , 0.01668176,
        0.        , 0.4574311 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.12

In [174]:
xb = output
index = Faiss().faiss(xb)
D,I = Faiss().query(index,xq)
I

array([[15, 66, 72]])

In [175]:
data[15]

'Jpmorgan Chase & Co'

In [172]:
data

['Apple Inc.',
 'Microsoft Corp',
 'Alphabet Inc.',
 'Amazon Com Inc',
 'Nvidia Corp',
 'Tesla, Inc.',
 'Berkshire Hathaway Inc',
 'Meta Platforms, Inc.',
 'Eli Lilly & Co',
 'Visa Inc.',
 'Taiwan Semiconductor Manufacturing Co Ltd',
 'Exxon Mobil Corp',
 'Unitedhealth Group Inc',
 'Walmart Inc.',
 'Novo Nordisk A S',
 'Jpmorgan Chase & Co',
 'Spdr S&P 500 Etf Trust',
 'Johnson & Johnson',
 'Mastercard Inc',
 'Lvmh Moet Hennessy Louis Vuitton',
 'Procter & Gamble Co',
 'Broadcom Inc.',
 'Latam Airlines Group S.A.',
 'Home Depot, Inc.',
 'Chevron Corp',
 'Oracle Corp',
 'Merck & Co., Inc.',
 'Abbvie Inc.',
 'Coca Cola Co',
 'Adobe Inc.',
 'Toyota Motor Corp/',
 'Pepsico Inc',
 'Costco Wholesale Corp /New',
 'Asml Holding Nv',
 'Bank Of America Corp /De/',
 'Cisco Systems, Inc.',
 'Alibaba Group Holding Ltd',
 'Salesforce, Inc.',
 'Shell Plc',
 'Astrazeneca Plc',
 'Novartis Ag',
 'Mcdonalds Corp',
 'Accenture Plc',
 'Thermo Fisher Scientific Inc.',
 'Mexican Economic Development Inc',
 '

In [96]:
output

array([[0.        , 0.3699596 , 0.5188751 ],
       [0.        , 0.561179  , 0.        ],
       [0.        , 0.12295088, 0.        ],
       [0.        , 0.68749493, 0.67374915],
       [0.        , 0.43349937, 0.75967026]], dtype=float32)

In [121]:
data = list(pd.read_csv("SEC-CompanyTicker.csv",index_col=0).companyName[:100])+["Apple"]

In [127]:
data[21]

'Broadcom Inc.'